In [ ]:
import os, time, librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as idp
import sklearn

from tqdm import tqdm
from tensorflow.keras import utils

In [ ]:
root_path = 'DataSource'
emotions_ref = [ 'Neutral', 'Calm', 'Happy', 'Sad', 'Angry', 'Fearful', 'Disgust', 'Surprised' ]

#### Output data set:

0. full file name
1. emotion

In [ ]:
def create_source_dataset(root_path): 
    
    current_time = time.time()    
    ds = []
    
    # get absolute root folder path
    root_abspath = os.path.join(os.path.abspath(os.getcwd()), root_path)    
    root_dirs = [os.path.join(root_abspath, x) for x in os.listdir(root_path)] 
    
    for root_dir in root_dirs:
        
        for file_src in os.listdir(root_dir):
            # 0 element
            file_path = os.path.join(root_dir, file_src)
            # 3 emotion 6..8
            emotion = int(file_src[6:8])
                
            ds.append([file_path, emotion ])
            
    assert len(ds) == 1440, 'it must be 1440 files in the source'
    
    print('Data source array is prepared - {} c'.format(time.time() - current_time))
    
    return df

In [ ]:
def get_features(y, sr):
    out = []
    
    out.append(np.mean(librosa.feature.rms(y = y)))
    out.append(np.mean(librosa.feature.spectral_centroid(y = y, sr = sr)))
    out.append(np.mean(librosa.feature.spectral_bandwidth(y = y, sr = sr)))
    out.append(np.mean(librosa.feature.spectral_rolloff(y = y, sr = sr))) 
    out.append(np.mean(librosa.feature.zero_crossing_rate(y)))
    
    for e in librosa.feature.mfcc(y = y, sr = sr):
        out.append(np.min(e))
        out.append(np.mean(e))
        out.append(np.max(e))
        
   
    for e in librosa.feature.chroma_stft(y = y, sr = sr):
        out.append(np.min(e))
        out.append(np.mean(e))
        out.append(np.max(e))

    return out

In [ ]:
def get_dataset(source):
    
    x_train = []
    y_train = []
    
    for item in tqdm(source):
        
        try:
            
            # read the audio file
            y, sr = read_audio(item[0])
            
            # append x_train item
            x_train.append(get_features(y, sr))
            y_train.append(utils.to_categorical(item[1], len(emotions_ref) + 1))
            
        except:
            
            print(f'Invalid object {item[0]}')
        
        
    return np.array(x_train), np.array(y_train)